### 如何实现聚焦爬虫(数据解析)
- 聚焦爬虫的编码流程
    1. 指定url
    2. 基于requests模块发起请求
    3. 获取响应对象中的数据
    4. 数据解析
    5. 进行持久化存储

### 如何实现数据解析
- 三种数据解析方式
    - 正则表达式
    - bs4
    - xpath
### 数据解析的原理
    - 进行标签定位
    - 获取定位好的标签里面的文本数据和属性值

In [2]:
# 如何爬取一张图片
import requests

# 1. 指定URL
url = "https://pic.qiushibaike.com/system/pictures/12206/122065848/medium/57J15TGANJTJBS2J.jpg"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"
}

# 2. 基于requests模块发起请求
response = requests.get(url=url, headers=headers)

# 3. 获取响应数据
page_content = response.content

# 4. 持久化存储
with open('./qiutu.jpg', 'wb') as f:
    f.write(page_content)


In [6]:
# 使用urllib模块下载图片
from urllib import request

url = "https://pic.qiushibaike.com/system/pictures/12206/122065811/medium/MA0W4J5QSG2KTQM0.jpg"

request.urlretrieve(url=url, filename="./yangmi.jpg")

('./yangmi.jpg', <http.client.HTTPMessage at 0x19ad4e98ba8>)

In [11]:
page_html = """
<div class="thumb"><a href="/article/122065919" target="_blank"><img src="//pic.qiushibaike.com/system/pictures/12206/122065919/medium/RX08K1QIBROHKKK1.jpg" alt="今天送趟孩子"></a></div>
"""

import re
ex = '<div class="thumb">.*?<img src="(.*?)" alt=.*?</div>'
print(re.findall(ex, page_html, re.S))

['//pic.qiushibaike.com/system/pictures/12206/122065919/medium/RX08K1QIBROHKKK1.jpg']


In [13]:
# 需求：爬取糗事百科指定页面的糗图，并将其保存到指定文件夹中
import os
import re
import requests
from urllib import request

url = "https://www.qiushibaike.com/pic/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"
}
ex = '<div class="thumb">.*?<img src="(.*?)" alt=.*?</div>'

# 在本地创建存储图片的文件夹
if not os.path.exists("./qiutu"):
    os.mkdir("./qiutu")

# 获取糗事百科糗图版块的页面全部数据
response = requests.get(url=url,headers=headers)
page_text = response.text

# 通过正则匹配解析图片URL
img_src_list = re.findall(ex, page_text, re.S)
print(img_src_list)

# 向图片URL发送请求，获取图片数据
for src in img_src_list:
    src = "https:" + src
    name = src.split("/")[-1]
    img_path = "./qiutu/" + name
    
    # 使用urllib发送请求获取并存储图片
    request.urlretrieve(url=src, filename=img_path)
    print("图片%s爬取成功!"%(name))


['//pic.qiushibaike.com/system/pictures/12206/122065997/medium/KYGTD0QQ0XDFEPAK.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065988/medium/16F9GPFHS3HAA9TU.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065976/medium/28HCVJCUHKTJG50U.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065962/medium/WOVCBBZ7XQQRYY8O.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065963/medium/PKJGP1CL5HAN7E0S.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065919/medium/RX08K1QIBROHKKK1.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065904/medium/5AGFSYO2XRWAJNXG.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065923/medium/JGHKIV5JFE8IPSC6.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065884/medium/432XEF5PB2WSWN71.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065852/medium/7BWOUXQTW36P5NU1.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065844/medium/QKZ5G7GQJZLB384Q.jpg', '//pic.qiushibaike.com/system/pictures/12206/122065843/medium/QN

### bs4数据解析
    - 环境安装:
        pip install bs4
        pip install lxml # 它是一个解析器,如果使用bs4进行数据解析,需要依靠lxml
    - BeautifulSoup解析原理:
        - 实例化一个BeautifulSoup对象,并将需要解析的页面源码数据加载到对象中
        - 使用该对象的相关属性和方法进行数据解析或提取
    - 如何实例化一个BeautifulSoup对象
        - 本地加载: 
            soup = BeautifulSoup("./test_page.html", "lxml")
        - 网络加载: 
            soup = BeautifulSoup(page_text, "lxml")
    - BeautifulSoup对象相关的属性和方法
        - soup.tagName 获取页面源码中遇到的第一个标签
        - 取属性: 返回的永远是一个单数
            - soup.tagName.attrs 取得标签里面所有的属性
            - soup.tagName.attrs[属性名] 取标签的单个属性
            - soup.tagName[属性名] 取标签的单个属性
        - 取文本:
            - soup.string: 只可以获取直系标签中的文本内容
            - soup.text: 获取标签下的所有文本内容
            - soup.get_text(): 获取标签下的所有文本内容
        - find() 返回的永远是一个单数
            - find(tagName): 通过标签名进行数据解析
            - find(tagName, 标签属性): 通过标签属性进行标签定位
        - find_all() 返回的永远是一个列表
            - 找到所有符合要求的标签
        - select 选择器 返回的永远是一个列表
            - 标签选择器, 类选择器, ID选择器, 层级选择器
            - 层级选择器:
                - 单层级: soup.select(".tang > ul > li")
                - 多层级: soup.select(".tang li")
        

In [19]:
from bs4 import BeautifulSoup

# 实例化一个BeautifulSoup对象,并将需要解析的页面源码数据加载到对象中
fp = open("./test_page.html", "r", encoding="utf-8")
soup = BeautifulSoup(fp, "lxml")
soup

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>测试bs4</title>
</head>
<body>
<div>
<p>百里守约</p>
</div>
<div class="song">
<p>李清照</p>
<p>王安石</p>
<p>苏轼</p>
<p>柳宗元</p>
<a href="http://www.song.com/" target="_self" title="赵匡胤">
<span>this is span</span>
        宋朝是最强大的王朝，不是军队的强大，而是经济很强大，国民都很有钱</a>
<a class="du" href="">总为浮云能蔽日,长安不见使人愁</a>
<img alt="" src="http://www.baidu.com/meinv.jpg"/>
</div>
<div class="tang">
<ul>
<li><a href="http://www.baidu.com" title="qing">清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村</a></li>
<li><a href="http://www.163.com" title="qin">秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山</a></li>
<li><a alt="qi" href="http://www.126.com">岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君</a></li>
<li><a class="du" href="http://www.sina.com">杜甫</a></li>
<li><a class="du" href="http://www.dudu.com">杜牧</a></li>
<li><b>杜小月</b></li>
<li><i>度蜜月</i></li>
<li><a href="http://www.haha.com" id="feng">凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘</a></li>
</ul>
</div>
</body>
</html>

In [21]:
# print(soup)
# print(soup.head)
# print(soup.body)
soup.a

<a href="http://www.song.com/" target="_self" title="赵匡胤">
<span>this is span</span>
        宋朝是最强大的王朝，不是军队的强大，而是经济很强大，国民都很有钱</a>

In [24]:
# soup.a.attrs
# soup.a.attrs["href"]
soup.a["title"]

'赵匡胤'

In [25]:
soup.p

<p>百里守约</p>

In [26]:
print(soup.p.string)
print(soup.p.text)
print(soup.p.get_text())

百里守约
百里守约
百里守约


In [29]:
soup.find("a")

<a href="http://www.song.com/" target="_self" title="赵匡胤">
<span>this is span</span>
        宋朝是最强大的王朝，不是军队的强大，而是经济很强大，国民都很有钱</a>

In [32]:
soup.find("div", class_="tang")

<div class="tang">
<ul>
<li><a href="http://www.baidu.com" title="qing">清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村</a></li>
<li><a href="http://www.163.com" title="qin">秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山</a></li>
<li><a alt="qi" href="http://www.126.com">岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君</a></li>
<li><a class="du" href="http://www.sina.com">杜甫</a></li>
<li><a class="du" href="http://www.dudu.com">杜牧</a></li>
<li><b>杜小月</b></li>
<li><i>度蜜月</i></li>
<li><a href="http://www.haha.com" id="feng">凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘</a></li>
</ul>
</div>

In [33]:
soup.find("div", class_="tang").string

In [34]:
soup.find("div", class_="tang").text

'\n\n清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村\n秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山\n岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君\n杜甫\n杜牧\n杜小月\n度蜜月\n凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘\n\n'

In [35]:
soup.find("div", class_="tang").get_text()

'\n\n清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村\n秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山\n岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君\n杜甫\n杜牧\n杜小月\n度蜜月\n凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘\n\n'

In [42]:
soup.find_all("div")

[<div>
 <p>百里守约</p>
 </div>, <div class="song">
 <p>李清照</p>
 <p>王安石</p>
 <p>苏轼</p>
 <p>柳宗元</p>
 <a href="http://www.song.com/" target="_self" title="赵匡胤">
 <span>this is span</span>
         宋朝是最强大的王朝，不是军队的强大，而是经济很强大，国民都很有钱</a>
 <a class="du" href="">总为浮云能蔽日,长安不见使人愁</a>
 <img alt="" src="http://www.baidu.com/meinv.jpg"/>
 </div>, <div class="tang">
 <ul>
 <li><a href="http://www.baidu.com" title="qing">清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村</a></li>
 <li><a href="http://www.163.com" title="qin">秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山</a></li>
 <li><a alt="qi" href="http://www.126.com">岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君</a></li>
 <li><a class="du" href="http://www.sina.com">杜甫</a></li>
 <li><a class="du" href="http://www.dudu.com">杜牧</a></li>
 <li><b>杜小月</b></li>
 <li><i>度蜜月</i></li>
 <li><a href="http://www.haha.com" id="feng">凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘</a></li>
 </ul>
 </div>]

In [36]:
soup.find_all(["div", "p"])

[<div>
 <p>百里守约</p>
 </div>, <p>百里守约</p>, <div class="song">
 <p>李清照</p>
 <p>王安石</p>
 <p>苏轼</p>
 <p>柳宗元</p>
 <a href="http://www.song.com/" target="_self" title="赵匡胤">
 <span>this is span</span>
         宋朝是最强大的王朝，不是军队的强大，而是经济很强大，国民都很有钱</a>
 <a class="du" href="">总为浮云能蔽日,长安不见使人愁</a>
 <img alt="" src="http://www.baidu.com/meinv.jpg"/>
 </div>, <p>李清照</p>, <p>王安石</p>, <p>苏轼</p>, <p>柳宗元</p>, <div class="tang">
 <ul>
 <li><a href="http://www.baidu.com" title="qing">清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村</a></li>
 <li><a href="http://www.163.com" title="qin">秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山</a></li>
 <li><a alt="qi" href="http://www.126.com">岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君</a></li>
 <li><a class="du" href="http://www.sina.com">杜甫</a></li>
 <li><a class="du" href="http://www.dudu.com">杜牧</a></li>
 <li><b>杜小月</b></li>
 <li><i>度蜜月</i></li>
 <li><a href="http://www.haha.com" id="feng">凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘</a></li>
 </ul>
 </div>]

In [38]:
soup.find_all("li", limit=3)

[<li><a href="http://www.baidu.com" title="qing">清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村</a></li>,
 <li><a href="http://www.163.com" title="qin">秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山</a></li>,
 <li><a alt="qi" href="http://www.126.com">岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君</a></li>]

In [48]:
soup.select(".song")

[<div class="song">
 <p>李清照</p>
 <p>王安石</p>
 <p>苏轼</p>
 <p>柳宗元</p>
 <a href="http://www.song.com/" target="_self" title="赵匡胤">
 <span>this is span</span>
         宋朝是最强大的王朝，不是军队的强大，而是经济很强大，国民都很有钱</a>
 <a class="du" href="">总为浮云能蔽日,长安不见使人愁</a>
 <img alt="" src="http://www.baidu.com/meinv.jpg"/>
 </div>]

In [39]:
soup.select(".tang > ul > li")

[<li><a href="http://www.baidu.com" title="qing">清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村</a></li>,
 <li><a href="http://www.163.com" title="qin">秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山</a></li>,
 <li><a alt="qi" href="http://www.126.com">岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君</a></li>,
 <li><a class="du" href="http://www.sina.com">杜甫</a></li>,
 <li><a class="du" href="http://www.dudu.com">杜牧</a></li>,
 <li><b>杜小月</b></li>,
 <li><i>度蜜月</i></li>,
 <li><a href="http://www.haha.com" id="feng">凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘</a></li>]

In [40]:
soup.select(".tang li")

[<li><a href="http://www.baidu.com" title="qing">清明时节雨纷纷,路上行人欲断魂,借问酒家何处有,牧童遥指杏花村</a></li>,
 <li><a href="http://www.163.com" title="qin">秦时明月汉时关,万里长征人未还,但使龙城飞将在,不教胡马度阴山</a></li>,
 <li><a alt="qi" href="http://www.126.com">岐王宅里寻常见,崔九堂前几度闻,正是江南好风景,落花时节又逢君</a></li>,
 <li><a class="du" href="http://www.sina.com">杜甫</a></li>,
 <li><a class="du" href="http://www.dudu.com">杜牧</a></li>,
 <li><b>杜小月</b></li>,
 <li><i>度蜜月</i></li>,
 <li><a href="http://www.haha.com" id="feng">凤凰台上凤凰游,凤去台空江自流,吴宫花草埋幽径,晋代衣冠成古丘</a></li>]

In [1]:
# 爬取三国演义上面的文章内容
import requests
from bs4 import BeautifulSoup

url = "http://www.shicimingju.com/book/sanguoyanyi.html"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"
}

page_text = requests.get(url=url, headers=headers).text
# print(page_text)

soup = BeautifulSoup(page_text, "lxml")
li_list = soup.select(".book-mulu li")

f = open("./sanguo.txt", "w", encoding="utf-8")
# 循环li_list列表, 获取li标签下的a标签下面的文本数据和url
for li in li_list:
    title = li.a.string
    detail_url = "http://www.shicimingju.com" + li.a["href"]
    
    # print(title, detail_url)
    # 向详情页的URL发送请求，获取详情页的文本数据
    detail_page_text = requests.get(url=detail_url, headers=headers).text
    detail_soup = BeautifulSoup(detail_page_text, "lxml")
    div_tag = detail_soup.find("div", class_="chapter_content")
    detail_text = div_tag.get_text()
    
    f.write(title + ":" + detail_text + "\n")

f.close()